# Asociate Knowledge Base

In this notebook, we will learn how to associate a Knowledge Base (KB) with an Amazon Bedrock Agent, allowing the agent to query contextual information using RAG (Retrieval-Augmented Generation) before generating its responses.

At this point, we have an agent capable of executing defined actions (e.g., calling external functions or services). In this section, we will expand its capabilities so that it can also:

Retrieve information from a previously configured Knowledge Base.

Combine factual knowledge with generative reasoning.

Respond with greater accuracy and context, using the RAG architecture.

Notebook content:

- Review of the existing agent.

- Association of the Knowledge Base with the agent.

- Execution of queries with semantic context.

In [17]:
import boto3
import os
import time
import json
import uuid

In [8]:
iam_client = boto3.client('iam')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
s3_client = boto3.client('s3')

session = boto3.session.Session()
region = session.region_name

In [12]:
# complete with yours
agent_id = 'XXXXXXXXXX'
alias_id = 'TSTALIASID' # it works with any agent

kb_id = 'XXXXXXXXXX'

model_sonnet_id = "eu.anthropic.claude-3-7-sonnet-20250219-v1:0"
model_arn = 'arn:aws:bedrock:xx-xxxx-x:XXXXXXXXXX:inference-profile/eu.anthropic.claude-3-7-sonnet-20250219-v1:0'

### Function

In [2]:
def invoke_agent_helper(query, session_id, agent_id, alias_id, session_state=None):
    """ Invokes an Amazon Bedrock Agent and returns the response. It allows you to
    send a user query to a specific agent and alias, maintaining conversation
    context across multiple turns using the session ID.

    Args:
        query (str): The user query or input message to send to the agent.
        session_id (str): Unique identifier for the conversation session.
        agent_id (str): The ID of the Bedrock Agent to invoke.
        alias_id (str): The alias ID associated with the agent.
        session_state (dict, optional): Previous session state to maintain context. Defaults to None.

    Returns:
        str: The final answer generated by the agent.
    """
    
    end_session: bool = False
    
    if not session_state:
        session_state = {}

    # invoke the agent API
    agent_response = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=alias_id,
        sessionId=session_id,
        enableTrace=True,
        endSession=end_session,
        sessionState=session_state
    )

    print(agent_response)

    event_stream = agent_response['completion']
    try:
        for event in event_stream:
            if 'chunk' in event:
                data = event['chunk']['bytes']
                print(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                return agent_answer
                
            elif 'trace' in event:
                print(json.dumps(event['trace'], indent=2, default=str))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        print("unexpected event.", event_stream)

## Knowledge Base

Remember how KB worked

In [ ]:
# retrieve and generate kb

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": "What information do you have?"
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": model_arn,
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

response['output']['text']

# Asociate KB with Agent

Our goal is for the agent to also have access to the KB. Remember to associate the necessary permissions with the role.

In [14]:
# asociate kb
response = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description='Access the knowledge base when customers ask about information.',
    knowledgeBaseId=kb_id,
    knowledgeBaseState='ENABLED'
)

In [15]:
# update agent 

response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)

# Invoke agent

Now our agent is able to perform queries on the kb.

In [18]:
session_id = str(uuid.uuid1())
query = "What is FP32?"

In [ ]:
response = invoke_agent_helper(query, session_id, agent_id, alias_id)

In [ ]:
response